In [1]:
import pandas as pd
import numpy as np
import cv2

In [2]:
df = pd.read_csv("../data/processed/dataframe.csv")

In [3]:
df.shape

(12433, 7)

In [4]:
df.head()

,Unnamed: 0,gameTime,label,position,team,visibility,url_local
0,0,1 - 00:00,PASS,680,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...
1,1,1 - 00:02,DRIVE,2400,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...
2,2,1 - 00:04,HIGH PASS,4080,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...
3,3,1 - 00:07,HEADER,7200,home,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...
4,4,1 - 00:08,HEADER,8920,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...


In [5]:
labels = df.label.unique()
labels

array(['PASS', 'DRIVE', 'HIGH PASS', 'HEADER', 'CROSS', 'SHOT',
       'BALL PLAYER BLOCK', 'OUT', 'THROW IN', 'PLAYER SUCCESSFUL TACKLE',
       'FREE KICK', 'GOAL'], dtype=object)

In [6]:
train_p, val_p, test_p = .7, .2, .1

In [7]:
def get_sets(num_indices, train_p=0.8):
    indices_total = np.arange(num_indices)
    np.random.shuffle(indices_total)
    train_indices = indices_total[: int(train_p*df.shape[0])]
    test_indices = indices_total[int(train_p*df.shape[0]):]
    # val_indices = indices_total[int(train_p*df.shape[0]): int(train_p*df.shape[0] + val_p*df.shape[0])]
    # test_indices = indices_total[int(train_p*df.shape[0] + val_p*df.shape[0]):]

    return (train_indices, val_indices, test_indices)

In [8]:
train_indices, val_indices, test_indices = get_sets(df.shape[0])

print(len(train_indices))
print(len(test_indices))
print(len(val_indices))

NameError: name 'val_indices' is not defined

In [ ]:
from tensorflow.keras.utils import to_categorical

to_categorical(np.array(["A", "B", "A"]))

ValueError: invalid literal for int() with base 10: 'A'

# Indices to Images

In [ ]:
df["low_image_path"] = df.url_local + "/" + df.position.astype(str) + "_224p.png"
df["high_image_path"] = df.url_local + "/" + df.position.astype(str) + "_720p.png"
df["low_image_path"][0],df["high_image_path"][0]

('england_efl/2019-2020/2019-10-01 - Blackburn Rovers - Nottingham Forest/680_224p.png',
 'england_efl/2019-2020/2019-10-01 - Blackburn Rovers - Nottingham Forest/680_720p.png')

In [ ]:
def load_image(url_path):
    img = cv2.imread("../data/processed/" + url_path)
    
    # normalize
    img = img / 255
    
    return img

s = load_image(df["low_image_path"][train_indices[0]])
s.shape

(224, 398, 3)

# Making Labels correct

In [ ]:
# !pip install scikit-learn


In [ ]:
df.head()

,Unnamed: 0,gameTime,label,position,team,visibility,url_local,low_image_path,high_image_path
0,0,1 - 00:00,PASS,680,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...
1,1,1 - 00:02,DRIVE,2400,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...
2,2,1 - 00:04,HIGH PASS,4080,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...
3,3,1 - 00:07,HEADER,7200,home,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...
4,4,1 - 00:08,HEADER,8920,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...


In [ ]:
labels_count = df.label.nunique()
label_matcher = {label: i for i, label in enumerate(df.label.unique())}
df["label_numeric"] = [label_matcher[r] for r in df.label]
df.head()
labels_onehot_total = np.eye(df.label.nunique())[df["label_numeric"]]
labels_onehot_total.shape

(12433, 12)

# How much memory

GB of each image = 0.000101444

Total Dataset: 12433

Total Size of Images = ~2 gb

In [ ]:
import os

os.stat("../data/processed/england_efl/2019-2020/2019-10-01 - Brentford - Bristol City/680_224p.png")

os.stat_result(st_mode=33188, st_ino=234187500833250812, st_dev=286833644, st_nlink=1, st_uid=1202824, st_gid=2626, st_size=101444, st_atime=1711473443, st_mtime=1710520793, st_ctime=1710520793)

# Putting Data Together

In [ ]:
# Note: This takes a really long time
train_features = []
load_image(df["low_image_path"][0])

for i, path in enumerate(df["low_image_path"][train_indices]):
    if i % 100 == 0:
        print(i)
    if i > 7500:
        break
    train_features.append(load_image(path))
train_labels = labels_onehot_total[train_indices]
train_features = np.array(train_features)
print(train_features.shape)
print(train_labels.shape)

0
100


KeyboardInterrupt: 

In [ ]:
for s in df["low_image_path"][train_indices]:
    print(s)

england_efl/2019-2020/2019-10-01 - Hull City - Sheffield Wednesday/640320_224p.png
england_efl/2019-2020/2019-10-01 - Brentford - Bristol City/955320_224p.png
england_efl/2019-2020/2019-10-01 - Middlesbrough - Preston North End/3538480_224p.png
england_efl/2019-2020/2019-10-01 - Brentford - Bristol City/4677840_224p.png
england_efl/2019-2020/2019-10-01 - Middlesbrough - Preston North End/1070880_224p.png
england_efl/2019-2020/2019-10-01 - Reading - Fulham/2622760_224p.png
england_efl/2019-2020/2019-10-01 - Leeds United - West Bromwich/4848280_224p.png
england_efl/2019-2020/2019-10-01 - Stoke City - Huddersfield Town/4068000_224p.png
england_efl/2019-2020/2019-10-01 - Reading - Fulham/3727520_224p.png
england_efl/2019-2020/2019-10-01 - Reading - Fulham/2234320_224p.png
england_efl/2019-2020/2019-10-01 - Brentford - Bristol City/3146520_224p.png
england_efl/2019-2020/2019-10-01 - Leeds United - West Bromwich/1638840_224p.png
england_efl/2019-2020/2019-10-01 - Brentford - Bristol City/178

In [ ]:
val_features = [load_image(path) for path in df["low_image_path"][train_indices]]
val_labels = labels_onehot_total[val_]
print(train_features.shape)
print(train_labels.shape)

,Unnamed: 0,gameTime,label,position,team,visibility,url_local,low_image_path,high_image_path,label_numeric
0,0,1 - 00:00,PASS,680,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,0
1,1,1 - 00:02,DRIVE,2400,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,1
2,2,1 - 00:04,HIGH PASS,4080,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,2
3,3,1 - 00:07,HEADER,7200,home,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,3
4,4,1 - 00:08,HEADER,8920,away,visible,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,england_efl/2019-2020/2019-10-01 - Blackburn R...,3


In [ ]:
import os

# Train Loop

In [1]:
from model_cnn import Model, CustomGenerator
import tensorflow as tf
from sklearn.utils import shuffle
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

2024-03-26 15:37:04.083227: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 15:37:05.033894: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 15:37:09.810368: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/current/lib:
2024-03-26 15:37:09.810908: W tensorflow/compiler/xla/stream_ex

In [3]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[]

In [4]:
df = pd.read_csv("../data/processed/dataframe.csv")
df["low_image_path"] = df.url_local + "/" + df.position.astype(str) + "_224p.png"
df["high_image_path"] = df.url_local + "/" + df.position.astype(str) + "_720p.png"


In [5]:
labels_count = df.label.nunique()
label_matcher = {label: i for i, label in enumerate(df.label.unique())}
df["label_numeric"] = [label_matcher[r] for r in df.label]

In [6]:
y_labels_one_hot = to_categorical(df["label_numeric"])


In [7]:
filenames_shuffled, y_labels_one_hot_shuffled = shuffle(df["low_image_path"], y_labels_one_hot)

np.save('y_labels_one_hot_shuffled.npy', y_labels_one_hot_shuffled)
np.save('filenames_shuffled.npy', filenames_shuffled)

In [8]:
# Used this line as our filename array is not a numpy array.
filenames_shuffled_numpy = np.array(filenames_shuffled)

X_train_filenames, X_val_filenames, y_train, y_val = train_test_split(
    filenames_shuffled_numpy, y_labels_one_hot_shuffled, test_size=0.2, random_state=1)

print(X_train_filenames.shape) # (3800,)
print(y_train.shape)           # (3800, 12)

print(X_val_filenames.shape)   # (950,)
print(y_val.shape)             # (950, 12)

# You can save these files as well. As you will be using them later for training and validation of your model.
np.save('X_train_filenames.npy', X_train_filenames)
np.save('y_train.npy', y_train)

np.save('X_val_filenames.npy', X_val_filenames)
np.save('y_val.npy', y_val)

(9946,)
(9946, 12)
(2487,)
(2487, 12)


In [9]:
BATCH_SIZE = 32

training_batch_generator = CustomGenerator(X_train_filenames, y_train, BATCH_SIZE)
validation_batch_generator = CustomGenerator(X_train_filenames, y_train, BATCH_SIZE)

In [10]:
model = Model(input_shape=(224, 396, 3))

2024-03-26 15:37:29.211757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model.fit(training_batch_generator, validation_batch_generator, 1)
model.save("cnn_1.keras")

311/311 [==============================] - 4890s 16s/step - loss: 2.2721 - accuracy: 0.2711 - val_loss: 1.6347 - val_accuracy: 0.3881


In [14]:
model.fit(training_batch_generator, validation_batch_generator, 1)
model.save("cnn_2.keras")

311/311 [==============================] - 4869s 16s/step - loss: 1.7298 - accuracy: 0.3688 - val_loss: 1.5501 - val_accuracy: 0.3918


In [15]:
model.fit(training_batch_generator, validation_batch_generator, 1)
model.save("cnn_3.keras")

311/311 [==============================] - 4872s 16s/step - loss: 1.6067 - accuracy: 0.3754 - val_loss: 1.5036 - val_accuracy: 0.4013


In [16]:
model.fit(training_batch_generator, validation_batch_generator, 1)
model.save("cnn_4.keras")

195/311 [=================>............] - ETA: 23:30 - loss: 1.5561 - accuracy: 0.3762

In [33]:
labels_onehot_total = np.eye(df.label.nunique())[df["label_numeric"]]
labels_onehot_total.shape

(12433, 12)

In [ ]:
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
EPOCHS = 10
BATCH_SIZE = 32
SPLIT = 0.8, .2

model = Model(input_shape=(224, 396, 3))

for e in range(EPOCHS):
    train_idx, _, test_idx = get_sets(df.shape[0])
    for i in range(0, len(train_idx), BATCH_SIZE):
        batch_idx = train_idx[i:i+BATCH_SIZE]
        
        X_train = np.array([load_image(df["low_image_path"][i]) for i in batch_idx])
        Y_train = labels_onehot_total[batch_idx]
        history = model.fit(X_train, Y_train)
        
    model.save("./cnn_2.keras")